In [2]:
import os
import numpy as np
import soundfile as sf
import librosa
import pandas

In [31]:
#define the sizes and timestamps of all silences to be inserted into track (based on parameters)
def define_silences_timestamps(start_wav, end_wav, total_wav, sr,
                               percent_silence, silences_nb,
                               silence_durations, silence_interval_min, 
                               rng = np.random.default_rng(42)):

    #choose a random number of silence amongst a list of possible number of silence, taking account of the total lenght of silence
    total_silence = round(total_wav*percent_silence/sr)
    possible_nb_silences = [x for x in silences_nb if (total_silence >= x*min(silence_durations)) and (total_silence <= x*max(silence_durations))]
    silence_nb = int(rng.choice(possible_nb_silences))

    #define n random durations of silences from a list of possible durations, taking account of the total lenght of silence
    last_silence_dur = 0
    while last_silence_dur not in silence_durations: 
        silences_length = rng.choice(silence_durations, size=silence_nb-1)
        last_silence_dur = total_silence - sum(silences_length)            
    silences_length = np.append(silences_length, last_silence_dur)

    #define the random start and the stop of each silence in the wav, taking account of the minimal duration between each silence
    silences_timestamps=[]
    for silence_duration in silences_length:
        while True:
            start_silence = rng.integers(low=start_wav*sr, high=end_wav-silence_duration*sr, size = 1, endpoint=True)
            stop_silence = start_silence+silence_duration*sr
            if len(silences_timestamps) > 0:
                correct_timestamp = []
                for (_, start, stop) in silences_timestamps: 
                    start_before_silence = stop_silence < (start - sr*silence_interval_min) 
                    start_after_silence = start_silence > (stop + sr*silence_interval_min)
                    correct_timestamp.append(start_before_silence + start_after_silence)
                    
                if np.prod(correct_timestamp):
                    silences_timestamps.append((silence_duration*sr, start_silence[0], stop_silence[0]))
                    break
                else:
                    pass
            else:
                silences_timestamps.append((silence_duration*sr, start_silence[0], stop_silence[0]))
                break
    
    return silences_timestamps

In [32]:
def insert_silences(wav, sr,
                    silences_timestamps,
                    fading_duration):
    #insert fading
    fadein = np.linspace(0,1,np.int32(fading_duration*sr))
    fadeout = np.linspace(1,0,np.int32(fading_duration*sr))
    wav[:np.int32(fading_duration*sr)] *= fadein
    wav[-np.int32(fading_duration*sr):] *= fadeout
    
    #insert silence in the track
    for (silence_duration, start, stop) in silences_timestamps:
        silence = np.zeros(np.int32(silence_duration))
        wav[start:stop] *= silence
    
    return wav

In [34]:
#define all parameters needed
path = '/home/maellef/git/MuteMusic/data/StimuliFinal/long'
outpath = "/home/maellef/git/MuteMusic/data/StimuliFinal/silenced"

#EDA ?
#during the instructions :
#explicit ; please try not to nod your head/tap/
#How well did you imagine  
#not at all / I could imagine perfectly 

#partially randomize order of tracks (5U/5F)

fading_duration = 1
intro_duration = 5
percent_silence = 0.25
silence_durations = [4,5,6]
silences_nb = [2,3,4]
silence_interval_min = 5 #3

silences_df = pandas.DataFrame()
rng = np.random.default_rng(2023)

for category in os.listdir(path):
    for file in os.listdir(os.path.join(path, category)):
        name, ext = os.path.splitext(file)
        if ext == '.wav':
            path_wav=os.path.join(path, category, file)
            wav, sr = librosa.load(path_wav, sr=None)
            print(name, len(wav)/sr)
            silences_timestamps = define_silences_timestamps(start_wav = intro_duration, end_wav=len(wav), total_wav=len(wav), sr=sr,
                                                             percent_silence=percent_silence, silences_nb=silences_nb,
                                                             silence_durations=silence_durations, silence_interval_min=silence_interval_min,
                                                             rng = rng)
            
            silenced_wav = insert_silences(wav, sr,silences_timestamps,fading_duration)
            print(len(silences_timestamps), silences_timestamps)
            
            if len(silences_timestamps) == 2:
                silence_data = pandas.DataFrame([{'category':category,
                                                  'track':file,
                                                  'S1_duration':silences_timestamps[0][0]/sr, 'S1_start':silences_timestamps[0][1], 'S1_stop':silences_timestamps[0][2],
                                                  'S2_duration':silences_timestamps[1][0]/sr, 'S2_start':silences_timestamps[1][1], 'S2_stop':silences_timestamps[1][2],
                                                 }])
            elif len(silences_timestamps) == 3:
                silence_data = pandas.DataFrame([{'category':category,
                                                  'track':file,
                                                  'S1_duration':silences_timestamps[0][0]/sr, 'S1_start':silences_timestamps[0][1], 'S1_stop':silences_timestamps[0][2],
                                                  'S2_duration':silences_timestamps[1][0]/sr, 'S2_start':silences_timestamps[1][1], 'S2_stop':silences_timestamps[1][2],
                                                  'S3_duration':silences_timestamps[2][0]/sr, 'S3_start':silences_timestamps[2][1], 'S3_stop':silences_timestamps[2][2],
                                                  }])
            else:
                silence_data = pandas.DataFrame([{'category':category,
                                                  'track':file,
                                                  'S1_duration':silences_timestamps[0][0]/sr, 'S1_start':silences_timestamps[0][1], 'S1_stop':silences_timestamps[0][2],
                                                  'S2_duration':silences_timestamps[1][0]/sr, 'S2_start':silences_timestamps[1][1], 'S2_stop':silences_timestamps[1][2],
                                                  'S3_duration':silences_timestamps[2][0]/sr, 'S3_start':silences_timestamps[2][1], 'S3_stop':silences_timestamps[2][2],
                                                  'S4_duration':silences_timestamps[3][0]/sr, 'S4_start':silences_timestamps[3][1], 'S4_stop':silences_timestamps[3][2],
                                                 }])
            silences_df = pandas.concat([silences_df, silence_data])
            

            new_name = f'{name}_silenced{ext}'
            new_path = os.path.join(outpath, category)
            os.makedirs(new_path, exist_ok=True)
            sf.write(file=os.path.join(new_path, new_name), data = silenced_wav, samplerate=sr)

output_file= os.path.join(outpath, 'silences_data.tsv')
silences_df.to_csv(output_file, sep='\t', index=False, header=True)
print(silences_df)        

Super_Mario_Bros_(NES)_Music_Overworld_Theme 59.0
3 [(240000, 610063, 850063), (192000, 1679267, 1871267), (288000, 2305597, 2593597)]
Super_Mario_Bros_3_Boss_Battle 57.0
3 [(192000, 697356, 889356), (192000, 1401540, 1593540), (288000, 2119862, 2407862)]
Final_Fantasy_7_Music_Aeriths_Theme 57.0
3 [(192000, 1431990, 1623990), (192000, 335400, 527400), (288000, 869031, 1157031)]
Super_Mario_Bros_(NES)_Music_Castle_Theme 47.0
2 [(288000, 890580, 1178580), (288000, 1553961, 1841961)]
Super_Mario_Bros_3_Castle_Theme 53.0
3 [(240000, 2202041, 2442041), (192000, 1570830, 1762830), (192000, 677482, 869482)]
Legend_of_Zelda_The_(NES)_Music_Overworld_Theme 50.0
3 [(192000, 1822604, 2014604), (192000, 689505, 881505), (192000, 1176625, 1368625)]
Super_Mario_Bros_3_Athletic_Theme 50.0
2 [(288000, 526615, 814615), (288000, 1833282, 2121282)]
Super_Mario_Bros_(NES)_Music_Underground_Theme 49.0
2 [(288000, 1272364, 1560364), (288000, 366380, 654380)]
Megalovania_Undertale 56.0
3 [(192000, 1029632, 1

In [48]:
#modification of specific files
path = '/home/maellef/git/MuteMusic/data/StimuliFinal/long/Lyrics/Make_Em_Laugh.wav'
outpath = '/home/maellef/git/MuteMusic/data/StimuliFinal/silenced/Lyrics/Make_Em_Laugh_silenced.wav'
silences_timestamps = [(4*sr, 475141, 667141), (4*sr, 2500975, 2692975), (4*sr, int(1378519.0-4*sr), int(1570519.0-4*sr)), (4*sr, int(1845411.0-4*sr), int(2037411.0-4*sr))]

print(silences_timestamps)
wav, sr = librosa.load(path, sr=None)
for dur, start, stop in silences_info:
    print(dur/sr, start/sr, stop/sr)

silenced_wav = insert_silences(wav, sr,silences_timestamps,fading_duration=1)
sf.write(file=outpath, data = silenced_wav, samplerate=sr)

[(192000, 475141, 667141), (192000, 2500975, 2692975), (192000, 1186519, 1378519), (192000, 1653411, 1845411)]
4.0 9.898770833333334 13.898770833333334
4.0 52.10364583333333 56.10364583333333
4.0 24.719145833333332 28.719145833333332
4.0 34.4460625 38.4460625


In [37]:
#test
path = '/home/maellef/git/MuteMusic/data/StimuliFinal/long'
silences_nb = [2,3,4]
silence_durations = [4,5,6]
l2, l3, l4 = [],[],[]

rng = np.random.default_rng(2022)

for category in os.listdir(path):
    for file in os.listdir(os.path.join(path, category)):
        name, ext = os.path.splitext(file)
        if ext == '.wav':
            path_wav=os.path.join(path, category, file)
            wav, sr = librosa.load(path_wav, sr=None)
            if len(wav)/sr < 46:
                print(file, len(wav)/sr)
            
            total_silence = round(len(wav)*0.25/sr)
            possible_nb_silences = [x for x in silences_nb if (total_silence >= x*min(silence_durations)) and (total_silence <= x*max(silence_durations))]
            silence_nb = int(rng.choice(possible_nb_silences))

            last_silence_dur = 0
            while last_silence_dur not in silence_durations: 
                silences_length = rng.choice(silence_durations, size=silence_nb-1)
                last_silence_dur = total_silence - sum(silences_length)            
            silences_length = np.append(silences_length, last_silence_dur)
            #print(file, len(wav)/sr, total_silence, silence_nb, silences_length)

Streets_Of_Rage_1_OST_Stage_1.wav 45.0
Shinobi_3_Return_Of_The_Ninja_Master_OST_Japonesque_(level_1_1).wav 44.0
Yakety_Sax(1983_version).wav 45.0
Hook_prologue.wav 44.0
The_Pink_Panther_Theme.wav 45.0
Another_Brick_In_The_Wall_(Part_1).wav 45.0


In [ ]:
#check sample rate for all files
wav, sr = librosa.load(path_wav, sr=None)

for dirpath, dirnames, files in os.walk(path):
    min_filename = ''
    wav_min = 3000000
    if len(files) > 0:
        for filename in files:
            name, ext = os.path.splitext(filename)
            if ext == '.wav':
                wav, wav_sr = librosa.load(os.path.join(dirpath, filename), sr=None)
                if len(wav) < wav_min:
                    wav_min = len(wav)
                    min_filename = filename
                if wav_sr != sr :
                    print(f'sr = {wav_sr} for {filename}')
                else :
                    pass
        print(f'check finished for {dirpath}, min duration is {wav_min/wav_sr}s for {min_filename}')
        